# 計算を実行

必要なライブラリのインポート

In [1]:
from PyFoam.Execution.BasicRunner import BasicRunner
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from os import getcwd, path
import subprocess  
import os

元ファイルをクローンします

In [2]:
# コピー元のケースディレクトリ
source_case = "pimpleFoam"

# ケースディレクトリの指定
case_dir = "pimpleFoam_run"

# 現在の作業ディレクトリを取得し、そこにコピー
destination_path = path.join(getcwd(), case_dir)

# SolutionDirectoryを使用してケースをコピー
template = SolutionDirectory(source_case).cloneCase(destination_path)

## PyFoamライブラリを使った計算実行（フォアグラウンド実行）

In [3]:
BasicRunner?

Init signature:
BasicRunner(
    argv=None,
    silent=False,
    logname=None,
    compressLog=False,
    lam=None,
    server=False,
    restart=False,
    noLog=False,
    logTail=None,
    remark=None,
    jobId=None,
    parameters=None,
    writeState=True,
    echoCommandLine=None,
)
Docstring:     
Base class for the running of commands

When the command is run the output is copied to a LogFile and
(optionally) standard-out

The argument list assumes for the first three elements the
OpenFOAM-convention:

<cmd> <dir> <case>

The directory name for outputs is therefor created from <dir> and
<case>

Provides some handle-methods that are to be overloaded for
additional functionality
Init docstring:
:param argv: list with the tokens that are the command line
if not set the standard command line is used
:param silent: if True no output is sent to stdout
:param logname: name of the logfile
:param compressLog: Compress the logfile into a gzip
:param lam: Information about a parallel ru

In [4]:
# blockMeshの実行
block_mesh_runner = BasicRunner(argv=["blockMesh", "-case", case_dir])
block_mesh_runner.start()

# pimpleFoamの実行
pimpleFoam_runner = BasicRunner(argv=["pimpleFoam", "-case", case_dir])
pimpleFoam_runner.start()

/*---------------------------------------------------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2406                                  |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
Build  : _be01ca78-20240625 OPENFOAM=2406 version=2406
Arch   : "LSB;label=32;scalar=64"
Exec   : blockMesh -case pimpleFoam_run
Date   : Nov 02 2024
Time   : 00:03:36
Host   : kamakiri
PID    : 292653
I/O    : uncollated
Case   : /home/kamakiri/OpenFOAM/kamakiri-v2406/run/17_gijyutusyo/004_PyFoam/pimpleFoam_run
nProcs : 1
trapFpe: Floating point exception trapping enabled (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time mo

{'lines': 18016,
 'uniqueid': '7830bfb1-9862-11ef-bd01-fbea59a9686b',
 'logfile': 'pimpleFoam_run/PyFoam.pimpleFoam.logfile',
 'casefullname': '/home/kamakiri/OpenFOAM/kamakiri-v2406/run/17_gijyutusyo/004_PyFoam/pimpleFoam_run',
 'casename': 'pimpleFoam_run',
 'solver': 'pimpleFoam',
 'solverFull': 'pimpleFoam',
 'commandLine': 'pimpleFoam -case pimpleFoam_run',
 'hostname': 'kamakiri',
 'remark': 'No remark given',
 'starttime': 'Sat Nov  2 00:03:36 2024',
 'time': 0.3,
 'stepNr': 1197,
 'lasttimesteptime': 'Sat Nov  2 00:04:12 2024',
 'keyboardInterrupt': False,
 'interrupted': False,
 'OK': True,
 'endSeen': True,
 'cpuTime': 36.480430000000005,
 'cpuUserTime': 35.140497,
 'cpuSystemTime': 1.339933,
 'wallTime': 36.638365268707275,
 'usedMemory': 141.01171875,
 'endtime': 'Sat Nov  2 00:04:12 2024',
 'fatalError': False,
 'fatalFPE': False,
 'fatalStackdump': False}

## Allrunの実行（フォアグラウンド実行）

AllrunスクリプトをpimpleFoam_runに保存します。

Allrun
```bash
#!/bin/bash

# クリーンアップ: 既存の結果ファイルやログを削除
foamCleanTutorials

# メッシュ生成: blockMeshを使って計算領域を定義
blockMesh > log.blockMesh

# シミュレーション実行: pimpleFoamソルバーを実行
pimpleFoam > log.pimpleFoam
```

以下のコマンドで権限を付与します。

`chmod -777 Allrun`

In [6]:
# Allrunをバックグラウンドで実行し、ログを非表示にする
def run_allrun(case_dir):
    with open(os.devnull, 'w') as devnull:
        subprocess.run([f'{os.getcwd()}/{case_dir}/Allrun'], cwd=case_dir, stdout=devnull, stderr=devnull)

# コピー先のディレクトリ
case_dir = 'pimpleFoam_run'

# Allrunを実行
run_allrun(case_dir)

## PyFoamによる計算実行（バックグラウンド）

In [7]:
from PyFoam.Execution.BasicRunner import BasicRunner
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
import subprocess
import os
import shutil
import threading #add

# コピー先のディレクトリ
case_dir = 'pimpleFoam_runBackground'

# チュートリアルケースをコピー
template.cloneCase(path.join(getcwd(), case_dir))

# Allrunをバックグラウンドで実行し、ログを非表示にする
def run_allrun(case_dir):
    # blockMeshの実行
    block_mesh_runner = BasicRunner(argv=["blockMesh", "-case", case_dir], silent=True, logname= "blockMesh")
    block_mesh_runner.start()

    # pimpleFoamの実行
    pimpleFoam_runner = BasicRunner(argv=["pimpleFoam", "-case", case_dir], silent=True, logname="pimpleFoam")
    pimpleFoam_runner.start()

# スレッドを作成してAllrunをバックグラウンドで実行
allrun_thread = threading.Thread(target=run_allrun, args=(case_dir,))
allrun_thread.start()

# 他の操作を続ける
print("Allrunをバックグラウンドで実行中...")

# 必要に応じてスレッドの終了を待つ
# allrun_thread.join()

Allrunをバックグラウンドで実行中...


In [25]:
print("test")

test


## Allrun実行（バックグラウンド）

In [30]:
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
import subprocess
import os
import shutil
import threading #add

# コピー先のディレクトリ
case_dir = 'pimpleFoam_runBackground'

# 結果を表示
print(template)

# Allrunをバックグラウンドで実行し、ログを非表示にする
def run_allrun(case_dir):
    with open(os.devnull, 'w') as devnull:
        subprocess.run([f'{os.getcwd()}/{case_dir}/Allrun'], cwd=case_dir, stdout=devnull, stderr=devnull)

# スレッドを作成してAllrunをバックグラウンドで実行
allrun_thread = threading.Thread(target=run_allrun, args=(case_dir,))
allrun_thread.start()

# 他の操作を続ける
print("Allrunをバックグラウンドで実行中...")

# 必要に応じてスレッドの終了を待つ
# allrun_thread.join()

Allrunをバックグラウンドで実行中...


In [31]:
print("test")

test


In [28]:
!cat pimpleFoam_runBackground/Allrun

#!/bin/bash

# Clean the case
foamCleanTutorials

# Generate the mesh
blockMesh

# Run the solver
pimpleFoam

# Post-processing (optional)
# e.g., reconstructPar, foamToVTK, etc.

# Notify the user
echo "Simulation completed."